In [8]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import plotly.graph_objects as go
import re
import time

import utils
from slurm import SlurmClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Slurm Queue

In [77]:
!sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
q1*          up 7-00:00:00      1   unk* h100-st-p548xlarge-4
q1*          up 7-00:00:00     28    mix h100-st-p548xlarge-[7,9-10,13-14,29,38,48-49,227-242,333,448-449]
q1*          up 7-00:00:00    363  alloc h100-st-p548xlarge-[0-3,5-6,11-12,15-28,30-37,41-47,79-226,264-319,325-332,334-447]
q1*          up 7-00:00:00     58   idle h100-st-p548xlarge-[8,39-40,50-78,243-263,320-324]


In [78]:
sl = SlurmClient()
sl.get_queue_summary()

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,PD,1
2,ar-ai-hipri,R,318
3,ar-ai-midpri,R,2
4,ar-ai-research-interns,R,3
5,ar-ai-voice-hipri,PD,92
6,ar-ai-voice-hipri,R,53


In [79]:
sl.get_recent_queue('all')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
47,7765,all,sft-eval,jiuhai,R,2024-08-08T23:30:57,1:58:03,16,7083
48,7761,all,sft-eval,jiuhai,R,2024-08-08T23:00:05,2:28:55,1,8935


In [80]:
sl.get_recent_queue('ar-ai-research-interns')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
53,7489,ar-ai-research-interns,bash,yfzhuang,R,2024-08-08T18:05:03,7:24:01,1,26641
54,7488,ar-ai-research-interns,bash,yfzhuang,R,2024-08-08T18:03:36,7:25:28,1,26728
52,7337,ar-ai-research-interns,jupyter,imzyc,R,2024-08-08T00:14:40,1-01:14:24,1,90864


In [81]:
sl.get_recent_queue('ar-ai-voice-hipri')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
67,7375,ar-ai-voice-hipri,21CC-MAIN-2023-14,arashe,R,2024-08-09T01:11:13,17:53,2,1073
68,7374,ar-ai-voice-hipri,20CC-MAIN-2023-14,arashe,R,2024-08-09T00:27:33,1:01:33,2,3693
71,7373,ar-ai-voice-hipri,19CC-MAIN-2023-14,arashe,R,2024-08-08T21:10:30,4:18:36,2,15516
72,7372,ar-ai-voice-hipri,18CC-MAIN-2023-14,arashe,R,2024-08-08T21:02:22,4:26:44,2,16004
73,7371,ar-ai-voice-hipri,17CC-MAIN-2023-14,arashe,R,2024-08-08T20:59:07,4:29:59,2,16199
74,7370,ar-ai-voice-hipri,16CC-MAIN-2023-14,arashe,R,2024-08-08T20:58:24,4:30:42,2,16242
75,7368,ar-ai-voice-hipri,14CC-MAIN-2023-14,arashe,R,2024-08-08T20:45:02,4:44:04,2,17044
77,7367,ar-ai-voice-hipri,13CC-MAIN-2023-14,arashe,R,2024-08-08T20:35:16,4:53:50,2,17630
78,7366,ar-ai-voice-hipri,12CC-MAIN-2023-14,arashe,R,2024-08-08T19:47:03,5:42:03,2,20523
79,7365,ar-ai-voice-hipri,11CC-MAIN-2023-14,arashe,R,2024-08-08T19:38:59,5:50:07,2,21007


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Metrics

# Launch

In [106]:
s_base = "/fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh"
s_vars = {
    "job-name": "mh19_336",
    "nodes": 32,
    "ntasks": 32
}

sbatch_vars_string = []
for k, v in sbatch_vars.items():
    sbatch_vars_string.append(f"--{k}={v}")
sbatch_vars_string = ' '.join(sbatch_vars_string)

job_id = utils.get_bash_output(
    f'sbatch --parsable {sbatch_vars_string} {sbatch_base}', print_cmd=True, print_output=True)
job_id = int(job_id)

sbatch --parsable --job-name=mh19_336 --nodes=32 --ntasks=32 /fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh
7906



In [119]:
!tail -f /fsx_0/user/tranx/output/slurm_logs/output_7906.txt

tail: cannot open '/fsx_0/user/tranx/output/slurm_logs/output_7906.txt' for reading: No such file or directory
tail: no files remaining


In [92]:
# https://docs.icer.msu.edu/Slurm_Environment_Variables/

In [91]:
s = utils.get_bash_output('ping google.com -c 3', print_output=True)

PING google.com (142.250.191.238) 56(84) bytes of data.
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=1 ttl=112 time=8.74 ms
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=2 ttl=112 time=8.30 ms
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=3 ttl=112 time=8.23 ms

--- google.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 8.230/8.421/8.736/0.224 ms



In [87]:
print(s)

PING google.com (142.250.191.238) 56(84) bytes of data.
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=1 ttl=112 time=8.62 ms
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=2 ttl=112 time=8.24 ms
64 bytes from ord38s32-in-f14.1e100.net (142.250.191.238): icmp_seq=3 ttl=112 time=9.76 ms

--- google.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2003ms
rtt min/avg/max/mdev = 8.238/8.873/9.760/0.646 ms

